## Divide and Conquer Algorithms



### Closest Points Naive

In [1]:
from collections import namedtuple
from itertools import combinations
from math import sqrt

Point = namedtuple('Point', 'x y')

def distance_squared(first_point, second_point):
    return (first_point.x - second_point.x) ** 2 + (first_point.y - second_point.y) ** 2

def minimum_distance_squared_naive(points):
    min_distance_squared = float("inf")
    for p, q in combinations(points, 2):
        min_distance_squared = min(min_distance_squared,
                                   distance_squared(p, q))
    return min_distance_squared

if __name__ == '__main__':
    input_n = int(input())
    input_points = []
    for _ in range(input_n):
        x, y = map(int, input().split())
        input_point = Point(x, y)
        input_points.append(input_point)
    print("{0:.9f}".format(sqrt(minimum_distance_squared_naive(input_points))))

3
1 4
2 6
7 9
2.236067977


### Closest Points
Find the closest pair of points in a set of **n≤10^5** points on a plane

In [16]:
# In large dataset with 10^5 points, mergesort algorithm gives result in almost 20 seconds.
# So, for sorting given points, we will use quicksort algorithm, which performs faster.
def quicksort(A):
    if len(A) <= 1:
        return A
    pivot = A[0]
    greater = [x for x in A[1:] if x > pivot]
    less = [x for x in A[1:] if x <= pivot]
    return quicksort(less) + [pivot] + quicksort(greater)

# To calculate distance between two points, we will use Pythagorean Theorem.
# So, distance = hypotenuse = sqrt((x1 - x2)^2 + (y1 - y2)^2).
def distance_calculator(first, second):
    return ((first[0] - second[0]) ** 2 + (first[1] - second[1]) ** 2) ** 0.5

# We will use recursive algorithm to seperate points into two parts and solve side-by-side.
# We will find minimum distance in left-hand side and right-hand side and get the minimum between them.
def minimum_left_right(A):
    minimum = float('inf')
    #print(A)
    if len(A) == 3:
        if distance_calculator(A[0], A[1]) <= distance_calculator(A[0], A[2]):
            A = A[0:2] + A[1:]
        else:
            A = A[0::2] + A[1:]
    if len(A) == 2:
        minimum = min(minimum, distance_calculator(A[0], A[1]))
        #print(minimum)
        return minimum
    mid = len(A) // 2
    X = minimum_left_right(A[:mid])
    Y = minimum_left_right(A[mid:])
    d = min(X, Y)
    return d

# We will look at the middle part because there may be closest points which are not included due to the seperation.
# So, we will filter the boundaries of points so that the distances from the x-axis of the mid-point to 
# the x-axes of filtered points are not bigger than the minimum distance found by "minimum_left_right" function.
def filter_mid(A, d):
    mid = len(A) // 2
    P = []
    for i in range(len(A)):
        if abs(A[mid][0] - A[i][0]) <= int(d):
            P.append(A[i])
    return P

# For filtered points, we will use mergesort algorithm to sort points by y-coordinates.
def mergesort(A):
    if len(A) <= 1:
        return A
    mid = len(A) // 2
    B = mergesort(A[:mid])
    C = mergesort(A[mid:])
    A_new = merge(B, C)
    return A_new

def merge(B, C):
    D = []
    while len(B) != 0 or len(C) != 0:
        if len(B) == 0 or len(C) == 0:
            break
        if B[0][1] <= C[0][1]:
            D.append(B[0])
            B = B[1:]
        else:
            D.append(C[0])
            C = C[1:]
    D = D + B + C
    return D

# Since filtered points are sorted by y-axis, we do not need to check every combination.
# Only 8 points will be enough to check whether closest points are there.
def final_test(A):
    d_prime = float('inf')
    for i in range(len(A)):
        for j in range(len(A)):
            if i != j:
                if abs(i - j) < 7:
                    d_prime = min(d_prime, distance_calculator(A[i], A[j]))
                else:
                    break
    return d_prime

# Finally, we will use all the functions we defined above and find the distance between closest points.
def minimum_distance_squared(points):
    assert 2 <= len(points) <= 10 ** 5
    points = quicksort(points)
    print('Points sorted by x-axis: ', points)
    distance = minimum_left_right(points)
    print('Minimum distance between points in right-hand side and left-hand side: ', distance)
    mid_points = filter_mid(points, distance)
    print('Points locating in middle part: ', mid_points)
    sort_y = mergesort(mid_points)
    print('Middle part sorted by y-axis: ', sort_y)
    mid_distance = final_test(sort_y)
    print('Minimum distance between points in middle part: ', mid_distance)
    minimum = min(distance, mid_distance)
    print('Distance between closest points: ', minimum)
    return minimum

In [10]:
# Constraints: 2 <= n <= 10^5 and -10^3 <= xi ; yi <= 10^3
import random
points = []
for i in range(100000):
    points.append([random.randint(-10**3, 10**3), random.randint(-10**3, 10**3)])

In [11]:
import time
start_time = time.time()
print(minimum_distance_squared(points))
print("--- %s seaconds ---" % (time.time() - start_time))

0.0
--- 0.659386396408081 seconds ---


In [12]:
# Constraints: 2 <= n <= 10^5 and -10^9 <= xi ; yi <= 10^9
import random
points = []
for i in range(50000):
    points.append([random.randint(-10**9, 10**9), random.randint(-10**9, 10**9)])

In [13]:
import time
start_time = time.time()
print(minimum_distance_squared(points))
print("--- %s seaconds ---" % (time.time() - start_time))

19443.762598838734
--- 0.29181575775146484 seaconds ---


In [14]:
# Constraints: 2 <= n <= 10^5 and 0 <= xi ; yi <= 10^9
import random
points = []
for i in range(50000):
    points.append([random.randint(0, 10**9), random.randint(0, 10**9)])

In [15]:
import time
start_time = time.time()
print(minimum_distance_squared(points))
print("--- %s seaconds ---" % (time.time() - start_time))

25111.342815548516
--- 0.28199148178100586 seaconds ---


In [17]:
points = [(6,9),(3,7),(15,4),(7,1),(9,15),(1,2),(3,17),(8,12),(5,3),(1,16),(2,3),(7,21)]
minimum_distance_squared(points)

Points sorted by x-axis:  [(1, 2), (1, 16), (2, 3), (3, 7), (3, 17), (5, 3), (6, 9), (7, 1), (7, 21), (8, 12), (9, 15), (15, 4)]
Minimum distance between points in right-hand side and left-hand side:  1.4142135623730951
Points locating in middle part:  [(5, 3), (6, 9), (7, 1), (7, 21)]
Middle part sorted by y-axis:  [(7, 1), (5, 3), (6, 9), (7, 21)]
Minimum distance between points in middle part:  2.8284271247461903
Distance between closest points:  1.4142135623730951


1.4142135623730951

In [18]:
points = [(7,9),(3,7),(4,4),(8,1),(10,15),(1,2),(3,17),(9,12),(4,3),(1,16),(2,3),(8,21)]
minimum_distance_squared(points)

Points sorted by x-axis:  [(1, 2), (1, 16), (2, 3), (3, 7), (3, 17), (4, 3), (4, 4), (7, 9), (8, 1), (8, 21), (9, 12), (10, 15)]
Minimum distance between points in right-hand side and left-hand side:  1.4142135623730951
Points locating in middle part:  [(3, 7), (3, 17), (4, 3), (4, 4)]
Middle part sorted by y-axis:  [(4, 3), (4, 4), (3, 7), (3, 17)]
Minimum distance between points in middle part:  1.0
Distance between closest points:  1.0


1.0